In [437]:
import pandas as pd
import os


In [440]:
#students pathway
students=os.path.join("raw_data","students_complete.csv")
pd_students =pd.read_csv(students, encoding= "UTF-8")


In [441]:
#schools pathway,rename schools
schools=os.path.join("raw_data","schools_complete.csv")
pd_schools =pd.read_csv(schools, encoding= "UTF-8")
pd_schools.rename(columns={"name":"school"},inplace=True)


In [420]:
#sum of budget
Total_Budget=pd_schools["budget"].sum()
Total_Students=pd_schools["size"].sum()
Total_Schools=pd_schools["School ID"].count()


In [442]:
# merge data sets 
merged= pd.merge(pd_students,pd_schools[["School ID","school","type","size","budget"]], on="school")


In [443]:
#collect data for all districts; math score, reading score, % pass for both 
read_avg=merged["reading_score"].mean()
math_avg=merged["math_score"].mean()
read_pass_percent_dist=round(len(merged.loc[merged["reading_score"] > 70])/Total_Students*100,2)
math_pass_percent_dist= round(len(merged.loc[merged["math_score"]>70])/Total_Students*100,2)
overall_passing_dist=(read_pass_percent_dist+math_pass_percent_dist)/2

# district summary 
#total schools, total budget, average math score ,average reading score , % passing math, % passing reading, % overall passing
dist_sum= {"Total Schools":[Total_Schools],"Total Budget":[Total_Budget],"Average Math Score":[math_avg],
           "Average Reading Score":[read_avg],"% Passing Math":read_pass_percent_dist,
           "% Passing Reading":math_pass_percent_dist,"% Overall Passing":overall_passing_dist}
dist_sum_df=pd.DataFrame(data= dist_sum)
dist_sum_df=dist_sum_df[["Total Schools","Total Budget","Average Math Score",
                         "Average Reading Score","% Passing Math","% Passing Reading","% Overall Passing"]]
dist_sum_df.head()


,Total Schools,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,24649428,78.985371,81.87784,82.97,72.39,77.68


In [444]:
# school summary; use merge DataFrame,drop unneeded colums 
school_summary_df= merged
#calculate averages per school 
school_summary_df=school_summary_df.groupby(['school','type', 'size', 'School ID','budget'])['math_score','reading_score'].mean()
school_summary_df=school_summary_df=school_summary_df.reset_index()
school_summary_df.rename(columns={"math_score":"Average Math Score","reading_score":"Average Reading Score"}, inplace=True)
school_summary_df["per student budget"]=school_summary_df["budget"]/school_summary_df["size"]
#calculate passing average per school
school_summary_df=school_summary_df.set_index(["school"])


# find count of passing reading grades
passing_count_dist_reading=merged.loc[merged["reading_score"]>70].groupby(["school"]).count()

passing_count_dist_reading["count"]=passing_count_dist_reading["budget"]
passing_count_dist_reading["count"]
# find count of passing math grades
passing_count_dist_math=merged.loc[merged["math_score"]>70].groupby(["school"]).count()
passing_count_dist_math["count"]=passing_count_dist_math["budget"]
# put passing % into school summary
school_summary_df["% Passing Reading"]=round((passing_count_dist_reading["count"]/school_summary_df["size"])*100,2)
school_summary_df["% Passing Math"]=round((passing_count_dist_math["count"]/school_summary_df["size"])*100,2)
school_summary_df["Overall Passing"]= round((school_summary_df["% Passing Reading"]
                                             +school_summary_df["% Passing Math"])/2,2)
school_summary_df


,type,size,School ID,budget,Average Math Score,Average Reading Score,per student budget,% Passing Reading,% Passing Math,Overall Passing
school,,,,,,,,,,
Bailey High School,District,4976,7,3124928,77.048432,81.033963,628.0,79.30,64.63,71.96
Cabrera High School,Charter,1858,6,1081356,83.061895,83.975780,582.0,93.86,89.56,91.71
Figueroa High School,District,2949,1,1884411,76.711767,81.158020,639.0,78.43,63.75,71.09
Ford High School,District,2739,13,1763916,77.102592,80.746258,644.0,77.51,65.75,71.63
Griffin High School,Charter,1468,4,917500,83.351499,83.816757,625.0,93.39,89.71,91.55
Hernandez High School,District,4635,3,3022020,77.289752,80.934412,652.0,78.19,64.75,71.47
Holden High School,Charter,427,8,248087,83.803279,83.814988,581.0,92.74,90.63,91.68
Huang High School,District,2917,0,1910635,76.629414,81.182722,655.0,78.81,63.32,71.06
Johnson High School,District,4761,12,3094650,77.072464,80.966394,650.0,78.28,63.85,71.06


In [445]:
# top five schools sorted 

school_summary_top_five=school_summary_df.sort_values("Overall Passing",ascending=False)

school_summary_top_five.head()

,type,size,School ID,budget,Average Math Score,Average Reading Score,per student budget,% Passing Reading,% Passing Math,Overall Passing
school,,,,,,,,,,
Wilson High School,Charter,2283,5,1319574,83.274201,83.989488,578.0,93.25,90.93,92.09
Pena High School,Charter,962,9,585858,83.839917,84.044699,609.0,92.20,91.68,91.94
Wright High School,Charter,1800,10,1049400,83.682222,83.955000,583.0,93.44,90.28,91.86
Cabrera High School,Charter,1858,6,1081356,83.061895,83.975780,582.0,93.86,89.56,91.71
Holden High School,Charter,427,8,248087,83.803279,83.814988,581.0,92.74,90.63,91.68


In [446]:
#bottom 5 schools 
school_summary_bottom_five=school_summary_df.sort_values("Overall Passing",ascending=True)

school_summary_bottom_five.head()


,type,size,School ID,budget,Average Math Score,Average Reading Score,per student budget,% Passing Reading,% Passing Math,Overall Passing
school,,,,,,,,,,
Rodriguez High School,District,3999,11,2547363,76.842711,80.744686,637.0,77.74,64.07,70.90
Huang High School,District,2917,0,1910635,76.629414,81.182722,655.0,78.81,63.32,71.06
Johnson High School,District,4761,12,3094650,77.072464,80.966394,650.0,78.28,63.85,71.06
Figueroa High School,District,2949,1,1884411,76.711767,81.158020,639.0,78.43,63.75,71.09
Hernandez High School,District,4635,3,3022020,77.289752,80.934412,652.0,78.19,64.75,71.47


In [447]:
# math score by grade scores_df=merged
scores_df=merged
scores_df=scores_df.groupby(['school','grade'])['math_score','reading_score'].mean()
math_score_df=scores_df.copy()
math_score_df.reset_index().pivot(index='school', columns='grade', values='math_score')



grade,10th,11th,12th,9th
school,,,,
Bailey High School,76.996772,77.515588,76.492218,77.083676
Cabrera High School,83.154506,82.765560,83.277487,83.094697
Figueroa High School,76.539974,76.884344,77.151369,76.403037
Ford High School,77.672316,76.918058,76.179963,77.361345
Griffin High School,84.229064,83.842105,83.356164,82.044010
Hernandez High School,77.337408,77.136029,77.186567,77.438495
Holden High School,83.429825,85.000000,82.855422,83.787402
Huang High School,75.908735,76.446602,77.225641,77.027251
Johnson High School,76.691117,77.491653,76.863248,77.187857


In [448]:

# reading score by grade
scores_df=merged
scores_df=scores_df.groupby(['school','grade'])['math_score','reading_score'].mean()
reading_score_df=scores_df.copy()
reading_score_df.reset_index().pivot(index='school', columns='grade', values='reading_score')

grade,10th,11th,12th,9th
school,,,,
Bailey High School,80.907183,80.945643,80.912451,81.303155
Cabrera High School,84.253219,83.788382,84.287958,83.676136
Figueroa High School,81.408912,80.640339,81.384863,81.198598
Ford High School,81.262712,80.403642,80.662338,80.632653
Griffin High School,83.706897,84.288089,84.013699,83.369193
Hernandez High School,80.660147,81.396140,80.857143,80.866860
Holden High School,83.324561,83.815534,84.698795,83.677165
Huang High School,81.512386,81.417476,80.305983,81.290284
Johnson High School,80.773431,80.616027,81.227564,81.260714


In [449]:
#Scores by School Spending
# make bins and labels:
school_spending_df=school_summary_df
school_spend_bins = [0,585,615,645,675]
school_spend_labels = ["<$585", "$585-615", "$615-645","$645-675"]
school_spending_df=school_spending_df.reset_index()
del school_spending_df["type"]
del school_spending_df["size"]
del school_spending_df["School ID"]
del school_spending_df["school"]
del school_spending_df["budget"]






In [450]:
# scores by school size 

school_size_df=school_summary_df
school_size_bins = [0,1000,2000,5000]
school_size_labels = ["Small (<1000) ", "Medium (1000-2000)", "Large (2000-5000)"]
school_size_df=school_size_df.reset_index()
del school_size_df["type"]
del school_size_df["School ID"]
del school_size_df["school"]
del school_size_df["budget"]

school_size_df["School_Size"] = pd.cut(school_size_df["size"], school_size_bins, labels=school_size_labels)



del school_size_df["size"]

school_size_df.reset_index()
school_size_df=school_size_df.groupby(school_size_df['School_Size'], as_index=True, sort=False, group_keys=True).mean()
school_size_df

,Average Math Score,Average Reading Score,per student budget,% Passing Reading,% Passing Math,Overall Passing
School_Size,,,,,,
Small (<1000),83.821598,83.929843,595.000,92.47000,91.15500,91.8100
Medium (1000-2000),83.374684,83.864438,605.600,93.24400,89.93000,91.5880
Large (2000-5000),77.746417,81.344493,635.375,80.18875,67.63125,73.9075


In [451]:
school_type_df=school_summary_df
#school_type_bins = [0,1000,2000,5000]
school_type_bins =["Charter","District"]
school_type_labels = ["Charter","District"]
school_type_df=school_type_df.reset_index()
del school_type_df["School ID"]
del school_type_df["school"]
del school_type_df["budget"]
del school_type_df["size"]
##school_type_df["School_Type"] = pd.cut(school_type_df["type"], school_type_bins ,labels=school_size_labels)
school_type_df=school_type_df.groupby(school_type_df['type'], as_index=True, sort=False, group_keys=True).mean()
school_type_df

,Average Math Score,Average Reading Score,per student budget,% Passing Reading,% Passing Math,Overall Passing
type,,,,,,
District,76.956733,80.966636,643.571429,78.322857,64.302857,71.31000
Charter,83.473852,83.896421,599.500000,93.051250,90.361250,91.70625
